## **Data processing**

**Data Cleaning Strategy**

Deduplication: We remove duplicate rows based on track_id to ensure uniqueness.

Missing Values: filling missing text fields (like artists) with "Unknown" to maintain data integrity without dropping rows.

Outlier Handling: filtering out tracks with unrealistic values (e.g., Tempo < 30 BPM or Duration < 30 seconds) that likely represent data errors or non-musical audio.


**Handling uutliers** 

Rule 1: Tempo must be realistic (greater than 30 BPM)

Rule 2: Duration must be meaningful (greater than 30 seconds / 30000ms)

Rule 3: Loudness usually shouldn't be positive (in digital audio) - purely a sanity check


**Feature Engineering**

To prepare the data for analysis and potential Machine Learning, we derive new features:

duration_minutes: Converted from milliseconds for better readability.

is_explicit_int: Converted boolean explicit column to integer (0/1) for compatibility with ML models.

popularity_segment: Grouped tracks into "Low", "Medium", and "High" popularity buckets to simplify categorical analysis.

In [0]:
from pyspark.sql.functions import col, count, when, round, lit
from pyspark.sql.window import Window
import pyspark.sql.functions as F

source_table = "default.spotify_bronze_partitioned"
target_table_silver = "default.spotify_silver"

df = spark.read.table(source_table)
print(f"Initial row count: {df.count()}")


# Removing duplicates
df_deduped = df.dropDuplicates(["track_id"])

# Handling missing values
df_cleaned = df_deduped.na.fill({
    "artists": "Unknown",
    "album_name": "Unknown",
    "track_name": "Untitled"
})

print(f"Row count after deduplication: {df_cleaned.count()}")

df_quality = df_cleaned.filter(
    (col("tempo") > 30) & 
    (col("duration_ms") > 30000)
)

dropped_count = df_cleaned.count() - df_quality.count()
print(f"Dropped {dropped_count} rows due to outlier/quality checks.")

# Feature Engineering
df_enriched = df_quality \
    .withColumn("duration_minutes", round(col("duration_ms") / 60000, 2)) \
    .withColumn("is_explicit_int", col("explicit").cast("integer")) \
    .withColumn("popularity_segment", 
        when(col("popularity") < 30, "Low")
        .when(col("popularity") < 70, "Medium")
        .otherwise("High")
    )

display(df_enriched.select("track_name", "duration_minutes", "is_explicit_int", "popularity_segment").limit(5))

df_enriched.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(target_table_silver)

print(f"Transformation complete. Data saved to {target_table_silver}")